- dc neuron example [here](https://nest-simulator.readthedocs.io/en/stable/auto_examples/brette_gerstner_fig_3d.html?highlight=dc_generator) [ and here](https://nest-simulator.readthedocs.io/en/stable/auto_examples/tsodyks_depressing.html?highlight=dc_generator#see-also)

1. Set params
2. Build network
3. Set patterns
4. Simulate

In [7]:
# nest
import nest
import nest.raster_plot
import nest.voltage_trace
# other
import pandas as pd
import os
import json
import time 
import timeit
import itertools
import numpy as np
from importlib import reload 
from itertools import permutations 
from itertools import combinations 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")
sns.set_palette("coolwarm")

# Parameters

In [8]:
def set_params(NE=960, NI=240, J_ex=6., J_in=-95., eps=0.1, stim_end=150., sub_fr=0.9, sup_fr=1.1,
              C_m=250.0, g_L=16.7, t_ref=2.0):
    '''
    Defines default parameter values and allows you to change the values for
    
    sim_params: general simulation parameters
    model_params: value specifics for the LIF neuron model
    synapse_params: parameter values using the STDP synapse
    
    '''

    sim_params = {
        
        'N_total':1200,
        'NE': 960, # Default: 4/5 are exci neurons, 1/5 are inh neurons
        'NI': 240,
        'eps':0.1,  # connection probability
        'J_in':-96.0,
        'J_ex':6.0,
        'resolution':0.1,  # temporal resolution of simulation in ms. Kumar2008: 0.1
        'delay':1.5,  # synaptic delay in the network
        'n_threads':8,
        'stim_start':0., # start applying current (dc)
        'stim_end':150., # end applying current (dc)
        'simtime':1000., # simulation time 
        'sub_fr':0.9, # subthreshold current amplitude
        'sup_fr':1.01, # suprathreshold current amplitude
    }

    model_params = {
        
        'C_m': 250.0,
        'E_L': -70.0,
        'E_ex': 0.0,
        'E_in': -80.0,
        'I_e': 0.0,
        'V_reset': -70.0,
        'V_th': -50.0,
        'g_L': 16.7,
        't_ref': 2.0, # Duration of refractory period
        'tau_syn_ex': 0.326, # Rise time of the excitatory synaptic alpha function
        'tau_syn_in': 0.326,
    }
    
    synapse_params_ex = {
        'model':'stdp_synapse',
        'lambda': 0.01,
        'alpha': 1.0,
        'delay': 1.5,
        'weight': 6.0,
        'Wmax': 100.0   
    }
    
    synapse_params_in = {
        'model':'stdp_synapse',
        'lambda': 0.01,
        'alpha': 1.0,
        'delay': 1.5,
        'weight':-95.0,
        'Wmax':-100.0  
    }
    
    return sim_params, model_params, synapse_params_ex, synapse_params_in

In [9]:
 def set_stim_amps(model_params):
    '''adjust the stimulation amplitudes to be slightly sub and supra threshold.'''
    I_th = (model_params['V_th'] - model_params['E_L']) * model_params['g_L']
    stim_amps = np.array([0.9, 1.01]) * I_th
    Asub = stim_amps[0]
    Asupra = stim_amps[1]
    print(Asub, Asupra)

In [10]:
sim_params, model_params, syn_params_ex, syn_params_in = set_params()

# Build network

In [11]:
# build network
nest.ResetKernel() 
nest.SetKernelStatus({'resolution': 0.1, 'print_time': False, 'local_num_threads': 8})

# ====== MAKE NEURON POPULATIONS =========
neuron_ids = nest.Create('iaf_cond_alpha', 1200, params = model_params)

#  ====== CREATE DC GENERATORS + SPIKE DETECT + MULTMETER =========
# create two independent dc generators 
dcgen_sub = nest.Create('dc_generator')
dcgen_supra = nest.Create('dc_generator')

# create spikedetector
spikedet = nest.Create('spike_detector')
# create multimeter that records the voltage
multimet = nest.Create('multimeter', params={'record_from': ['V_m']})

# set status voltage meter with a recording interval 
nest.SetStatus(multimet, params={'interval':1.})

# set status spikedetector
nest.SetStatus(spikedet, params={"withgid": True, "withtime": True})

# ====== PARAMETERISE SYNAPSES AND CONNECT NEURONS =========
 # define network connectivity
conn_dict = {'rule': 'pairwise_bernoulli', 'p': 0.1} 

static_ex_params = {'model':'static_synapse','weight':6.0, 'delay':1.5}
static_in_params = {'model':'static_synapse','weight':-96.0, 'delay':1.5}

# make connections between the two populations
# from exc neurons to all neurons
nest.Connect(neuron_ids[:960], neuron_ids, conn_dict, syn_params_ex)
# from interneurons to all neurons
nest.Connect(neuron_ids[240:], neuron_ids, conn_dict, static_in_params)

# ====== CONNECT TO DEVICES =========
nest.Connect(neuron_ids, spikedet)
nest.Connect(multimet, neuron_ids)



# ===== FOR =======
pattern = [1, 1, 1, 1, 0, 0, 0, 0]
n_chunk = int(1200/(len(pattern)))

indices = list(range(1200))
# divide neuron indices in N chunks
make_n_chunks = [indices[i:i + 150] for i in range(0, len(indices), n_chunk)]

# make lists for the neurons that should receive a sub and supra amplitude
sub_amp_ids = []
supra_amp_ids = []

# divide the indices to sub or supra based on the given pattern
for i in range(len(pattern)):
    
    if pattern[i] == 0:
        sub_amp_ids.append(make_n_chunks[i])

    elif pattern[i] == 1:
        supra_amp_ids.append(make_n_chunks[i])

# need to convert list of tuples to one tuple for nest
neurons_sub = tuple([item for sublist in sub_amp_ids for item in sublist])
neurons_supra = tuple([item for sublist in supra_amp_ids for item in sublist])

#====== DC GENERATOR ======
nest.SetStatus(dcgen_sub, params = {'amplitude': 300.6, 'start':0., 'stop':100.})
nest.SetStatus(dcgen_supra, params = {'amplitude': 337.34, 'start':0., 'stop':100.})

# connect dc_generators to neurons
nest.Connect(dcgen_sub, neurons_sub)
nest.Connect(dcgen_supra, neurons_supra)

NESTErrors.IllegalConnection: ('IllegalConnection in Connect_g_g_D_D: Creation of connection is not possible because:\nWe do not allow to connect a device to a global receiver at the moment', 'IllegalConnection', <SLILiteral: Connect_g_g_D_D>, ': Creation of connection is not possible because:\nWe do not allow to connect a device to a global receiver at the moment')

# Set a pattern

In [44]:
pattern = [1, 1, 1, 1, 0, 0, 0, 0]

## 

In [49]:
n_chunk = int(1200/(len(pattern)))

indices = list(range(1200))
# divide neuron indices in N chunks
make_n_chunks = [indices[i:i + 150] for i in range(0, len(indices), n_chunk)]

# make lists for the neurons that should receive a sub and supra amplitude
sub_amp_ids = []
supra_amp_ids = []

# divide the indices to sub or supra based on the given pattern
for i in range(len(pattern)):
    
    if pattern[i] == 0:
        sub_amp_ids.append(make_n_chunks[i])

    elif pattern[i] == 1:
        supra_amp_ids.append(make_n_chunks[i])

# need to convert list of tuples to one tuple for nest
neurons_sub = tuple([item for sublist in sub_amp_ids for item in sublist])
neurons_supra = tuple([item for sublist in supra_amp_ids for item in sublist])

In [50]:
# #====== DC GENERATOR ======
# nest.SetStatus(dcgen_sub, params = {'amplitude': 300.6, 'start':0., 'stop':100.})
# nest.SetStatus(dcgen_supra, params = {'amplitude': 337.34, 'start':0., 'stop':100.})

# # connect dc_generators to neurons
# nest.Connect(dcgen_sub, neurons_sub)
# nest.Connect(dcgen_supra, neurons_supra)

# Simulate

In [ ]:
# ====== SIMULATE =========
# simulate for a certain time period (ms)
nest.Simulate(self.simtime)

# spike detector data
spike_times = nest.GetStatus(spikedet, 'events')[0]['times']
spike_neurons = nest.GetStatus(spikedet, 'events')[0]['senders']

# multimeter data
events = nest.GetStatus(multimet)[0]['events']
etimes = events['times']

return spikedet, multimet, events, etimes, spike_times, spike_neurons